In [35]:
import pandas as pd
import gc

import requests
import datetime
from bs4 import BeautifulSoup

# 휴일 표시

In [36]:
def print_whichday(year, month, day) :
    r = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    aday = datetime.date(year, month, day)
    bday = aday.weekday()
    return r[bday]

def get_request_query(url, operation, params, serviceKey):
    import urllib.parse as urlparse
    params = urlparse.urlencode(params)
    request_query = url + '/' + operation + '?' + params + '&' + 'serviceKey' + '=' + serviceKey
    return request_query

In [7]:
df_info  = pd.read_csv('data/data_info.csv')
df_train = pd.read_csv('data/train.csv')
df_test  = pd.read_csv('data/test.csv')

In [32]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'data/{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [8]:
df_train.head()

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,0,1,106,지방도1112호선,0,0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,0,2,103,일반국도11호선,0,0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,0,2,103,일반국도16호선,0,0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,0,2,107,태평로,0,0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,0,2,103,일반국도12호선,0,0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


In [9]:
df_test.head()

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,...,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted
0,TEST_000000,20220825,목,17,0,3,107,연삼로,0,0,...,0.0,0,산지2교,33.499427,126.541298,없음,제주은행사거리,33.500772,126.543837,있음
1,TEST_000001,20220809,화,12,0,2,103,일반국도12호선,0,0,...,0.0,3,중문입구,33.258507,126.427003,없음,관광단지입구,33.258119,126.415840,없음
2,TEST_000002,20220805,금,2,0,1,103,일반국도16호선,0,0,...,0.0,0,도순3교,33.258960,126.476508,없음,도순2교,33.259206,126.474687,없음
3,TEST_000003,20220818,목,23,0,3,103,일반국도11호선,0,0,...,0.0,0,아라주공아파트,33.473494,126.545647,없음,인다마을,33.471061,126.545467,없음
4,TEST_000004,20220810,수,17,0,3,106,번영로,0,0,...,0.0,0,부록교 시종점,33.501477,126.569223,없음,봉개교 시종점,33.496863,126.581230,없음


In [10]:
len(df_train)

4701217

In [26]:
df_info.loc[11, '변수명'] = 'height_restricted'

In [27]:
for i, row in df_info.iterrows():
    if i != 0:
        print (row['변수명'])
        print (df_train[row['변수명']].unique())

base_date
[20220623 20220728 20211010 20220311 20211005 20210913 20220106 20211213
 20211004 20211208 20220724 20211229 20220507 20220203 20220501 20220701
 20211001 20220319 20220210 20220119 20210907 20220205 20210927 20220107
 20210903 20220116 20220612 20210928 20211028 20211018 20220522 20211205
 20220523 20220525 20220702 20220610 20210917 20220201 20211019 20220306
 20210910 20210920 20211227 20210921 20220213 20220215 20220530 20220208
 20220721 20220608 20220114 20220115 20220512 20211030 20211024 20211202
 20220125 20220104 20220131 20220317 20220214 20211214 20220220 20211023
 20220606 20220529 20220221 20220602 20211011 20211015 20211002 20220607
 20220508 20211105 20220102 20210909 20210929 20210918 20220207 20210901
 20211226 20210906 20211102 20220209 20220218 20220219 20211101 20220505
 20220118 20220628 20211104 20211228 20220304 20220223 20220331 20220322
 20220117 20211007 20220212 20211203 20211008 20220731 20211013 20210902
 20211017 20220110 20220629 20220217 2021

[126.66233499 126.52624007 126.3621475  126.56622824 126.33015179
 126.50606877 126.26937756 126.44226579 126.76940888 126.49293482
 126.56776621 126.62548581 126.55097847 126.62386724 126.54270267
 126.48640854 126.38533664 126.41007676 126.90279286 126.51204601
 126.68595435 126.64482115 126.51057382 126.48900572 126.55431384
 126.47977252 126.44424604 126.35498679 126.60416217 126.40912885
 126.69611703 126.32793758 126.55448279 126.6958095  126.56580539
 126.2630415  126.49636795 126.49015329 126.54564685 126.26302994
 126.35034408 126.56525648 126.45638448 126.35308836 126.57803612
 126.61993324 126.35678811 126.39683669 126.8322542  126.40415278
 126.5726991  126.54731395 126.48350657 126.3725803  126.52350273
 126.79364939 126.50439627 126.405338   126.53167892 126.54546676
 126.42397057 126.50566369 126.56609211 126.55617317 126.54083157
 126.50733329 126.56819776 126.76964986 126.41844275 126.40725898
 126.35063036 126.72095493 126.25731312 126.48230711 126.56440729
 126.71671

['제3교래교' 'KAL사거리' '상창육교' '서현주택' '애월입구' '서호2차현대맨션203동' '나동' '외도천교' '제2가시교'
 '노루생이삼거리' '정방수퍼' '버으내교차로' '서흥동사무소' '송목교' '제2아라교' '흘천5교' '장전1교차로' '고성교차로'
 '송내교차로' '동산교' '농업용관정' '뱅듸왓교차로' '삼주연립101동' '제4산록교' '서홍교' '도순교' '어시천교' '금악'
 '명도암교차로' '상귀교차로' '남수교' '고산전화국' '별도교' '팔도철물건재' '명월교' '신제주초교입구오거리'
 '삼무공원사거리' '아라주공아파트' '농협주유소' '고하상로교차로' '부부식당' '해안육교' '오당빌레길1' '삼양교(서측)'
 '수장교' '상창교차로' '경마장교' '난산입구' '미수2교' '다이소제주화북점' '염광아파트' '남녕고앞사거리' '창천교'
 '수모루' '성읍삼거리' '제5산록교' '오장교' '인다마을' '제3고성교' '시청입구2' 'BYC' '거로사거리' '용당교차로'
 '의료보험조합' '대흥개발(주)' '성읍2리' '일호유리' '유수암교차로' '전표1교' '진은교' '한림어촌계' '제주일고'
 '수복당' '천주교회' '제2성읍교' '봉좌교' '제주삼다수' '구엄교' '더럭초교차로' '한라교' '신양약방' '월림'
 '비자림입구' '중문입구' '해안마을' '상효2교' '선경오피스텔' '어리목입구' '김약국' '세화입구' '원일공사' '봉성교차로'
 '광령2교차로' '서흥사거리' '월드컵경기장' '오돌또기' '색달교' '어승생삼거리' '한북교' '우성사료' '연북5교'
 '제4교래교' '제2의귀교' '국립제주박물관' '수산2리입구' '노형오거리' '광전출하반' '영실교' '금능사거리' '김녕교차로'
 '용흥교(동측)' '창고천교' '제1성읍교' '서귀포시청' '경동보일러' '오라오거리' '하례입구삼거리' '외도교' '성읍교차로'
 '명월입구' '쉬는팡교차로' '제1해안교' '한대교' '삼양동사무소' '제주민속식품가공공장' '한남투자' '월

In [25]:
df_train

Index(['id', 'base_date', 'day_of_week', 'base_hour', 'road_in_use',
       'lane_count', 'road_rating', 'road_name', 'multi_linked',
       'connect_code', 'maximum_speed_limit', 'vehicle_restricted',
       'weight_restricted', 'height_restricted', 'road_type',
       'start_node_name', 'start_latitude', 'start_longitude',
       'start_turn_restricted', 'end_node_name', 'end_latitude',
       'end_longitude', 'end_turn_restricted', 'target'],
      dtype='object')